In [13]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

In [25]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_293bd6374438497789ef95210e06385a = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='DgLrDb-7cje9BzPGC-qU25CCc7BIn7mKgFeqsrRzkGZx',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_293bd6374438497789ef95210e06385a.get_object(Bucket='torontopostalcodes-donotdelete-pr-8hik5btg1pjike',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df1 = pd.read_csv(body,index_col='Postal Code')
df1=df1.loc[['M5G','M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A']].reset_index()
df1=df1.rename(columns={'Postal Code':'Postcode'})
df1.head()


,Postcode,Latitude,Longitude
0,M5G,43.657952,-79.387383
1,M2H,43.803762,-79.363452
2,M4B,43.706397,-79.309937
3,M1J,43.744734,-79.239476
4,M4G,43.709060,-79.363452


In [26]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]
df2= pd.read_html(str(table))[0]
df2 =df2.groupby(['Postcode','Borough'])['Neighbourhood'].agg([('Neighbourhood', ', '.join)]).reset_index()
df2.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [28]:
df3 = pd.merge(df1,df2, on='Postcode', how='inner')
df3=df3[['Postcode','Borough','Neighbourhood','Latitude','Longitude']]
df3

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
